In [ ]:
import numpy as np
import torch
import os
import cv2
from PIL import Image

import clip
import wav2clip
from moviepy.audio.AudioClip import AudioArrayClip
import moviepy.video.io.VideoFileClip

In [ ]:
def get_file_list(dir_path):
    file_list = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            file_list.append(path)
            
    return file_list

In [ ]:
def extract_avg_frame(video_path):
    video = cv2.VideoCapture(video_path)

    success = True
    count = 0
    while success:
        success, frames = video.read()
        sum_frame = sum_frame + frames
        count = count + 1

    mean_frame = sum_frame/count
    return mean_frame


In [ ]:
def extract_audio(video_path):
    video = moviepy.video.io.VideoFileClip(video_path)
    audio = video.audio
    return audio

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

dir_path = r'C:\\Masters\\semester_3\\Project_1\\data\\ERB3_Stimuli'
file_list = get_file_list(dir_path)
img_features = []

for file in file_list:
    #Extract a mean image for every video clip
    image = extract_avg_frame(file)
    audio = extract_audio(file)
    model_vid, preprocess = clip.load("ViT-B/32", device=device)
    model_aud = wav2clip.get_model()
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model_vid.encode_image(image)
        audio_embeddings = wav2clip.embed_audio(audio, model_aud)
        img_features.append(image_features)